In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np
import optuna

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [20]:
train_loader = DataLoader(dset.MNIST(root='data', train=True, download=True, transform=T.ToTensor()), batch_size=64, shuffle=True)

test_loader = DataLoader(dset.MNIST(root='data', train=False, download=True, transform=T.ToTensor()), batch_size=64, shuffle=True)

In [15]:
class Net(nn.Module):
    def __init__(self, hidden_layer):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, hidden_layer)
        self.fc2 = nn.Linear(hidden_layer, 10)
        
    def forward(self, x):
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [16]:
model = Net(64)
model.to(device)

criterian = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [24]:
import time

epoch = 10

start_time = time.time()

data = train_loader.dataset.train_data.to(device).float()
target = train_loader.dataset.train_labels.to(device)
    
for i in range(epoch):
    optimizer.zero_grad()
    output = model(data)
    
    loss = criterian(output, target)
    loss.backward()
    optimizer.step()
    print("Epoch: ", i, "Loss: ", loss.item())
    
end_time = time.time()
print("Time: ", end_time - start_time)

Epoch:  0 Loss:  34.063541412353516
Epoch:  1 Loss:  169.40570068359375
Epoch:  2 Loss:  184.0341033935547
Epoch:  3 Loss:  173.37156677246094
Epoch:  4 Loss:  138.94383239746094
Epoch:  5 Loss:  111.84375762939453
Epoch:  6 Loss:  101.7469482421875
Epoch:  7 Loss:  84.93761444091797
Epoch:  8 Loss:  51.4688720703125
Epoch:  9 Loss:  28.604713439941406
Time:  0.8943591117858887


In [ ]:
with torch.no_grad():
	correct = 0
	total = 0
	for data, target in test_loader:
		data, target = data.to(device), target.to(device)
		outputs = model(data)
		predicted = torch.max(outputs.data, 1)[1]
		total += target.size(0)
		correct += (predicted == target).sum().item()

	print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} % ')

Accuracy of the network on the 10000 test images: 39.24 % 


### Tunning

In [ ]:
def objective(trial):
    hidden_size = trial.suggest_int('hidden_size', 128, 512) # suggest_categorical([values]) → pick from a fixed list
    learning_rate = trial.suggest_float('lr', 1e-4, 1e-1, log=True)

    transform = T.Compose([T.ToTensor()])
    train_loader = DataLoader(dset.MNIST(
        'data', train=True, download=False, transform=transform), batch_size=32, shuffle=True)

    model = Net(hidden_size)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    model.train()
    for epoch in range(1):
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

    return loss.item()

In [40]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)
print("Best parameters : ", study.best_params)

[I 2025-08-21 15:23:12,864] A new study created in memory with name: no-name-c5946653-63e6-47d8-b6fd-4edf633ac270
[I 2025-08-21 15:23:23,960] Trial 0 finished with value: 0.2864382863044739 and parameters: {'hidden_size': 469, 'lr': 0.00025684489753747956}. Best is trial 0 with value: 0.2864382863044739.
[I 2025-08-21 15:23:32,971] Trial 1 finished with value: 0.1618974655866623 and parameters: {'hidden_size': 315, 'lr': 0.0002547368804265993}. Best is trial 1 with value: 0.1618974655866623.
[I 2025-08-21 15:23:42,763] Trial 2 finished with value: 0.12343405187129974 and parameters: {'hidden_size': 309, 'lr': 0.023790261912498403}. Best is trial 2 with value: 0.12343405187129974.
[I 2025-08-21 15:23:52,935] Trial 3 finished with value: 0.05030718818306923 and parameters: {'hidden_size': 458, 'lr': 0.0018221988714715515}. Best is trial 3 with value: 0.05030718818306923.
[I 2025-08-21 15:24:04,528] Trial 4 finished with value: 0.5160250067710876 and parameters: {'hidden_size': 489, 'lr':

Best parameters :  {'hidden_size': 458, 'lr': 0.0018221988714715515}


In [43]:
X_test = test_loader.dataset.test_data.to(device).float()
y_test = test_loader.dataset.test_labels.to(device)

# Evaluate loss
criterion = nn.CrossEntropyLoss()
test_loss = criterion(model(X_test), y_test).item()

# Evaluate accuracy
with torch.no_grad():
    y_pred = torch.argmax(model(X_test), dim=1)
    acc = (y_pred == y_test).float().mean().item()
print("Test Loss:", test_loss)
print("Test Accuracy:", acc * 100, "%")


Test Loss: 17.14533042907715
Test Accuracy: 61.08999848365784 %


c:\ProgramData\anaconda3\envs\torch-gpu\lib\site-packages\torchvision\datasets\mnist.py:81: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
c:\ProgramData\anaconda3\envs\torch-gpu\lib\site-packages\torchvision\datasets\mnist.py:71: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
